In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### Import dataframe

In [2]:
df = pd.read_csv('zomato.csv')
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


#### checking for possible missing values in the data

In [3]:
#get the summation of missing values
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [4]:
#drop missing values in the location column row-wise
df.dropna(axis = 'index', subset = ['location'], inplace = True)

In [5]:
#confirm that all missing values in the location column are removed
#this is because location information is central to our analysis
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

#### Data wrangling

In [6]:
#to know the names of unique locations
df['location'].unique()

array(['Banashankari', 'Basavanagudi', 'Mysore Road', 'Jayanagar',
       'Kumaraswamy Layout', 'Rajarajeshwari Nagar', 'Vijay Nagar',
       'Uttarahalli', 'JP Nagar', 'South Bangalore', 'City Market',
       'Nagarbhavi', 'Bannerghatta Road', 'BTM', 'Kanakapura Road',
       'Bommanahalli', 'CV Raman Nagar', 'Electronic City', 'HSR',
       'Marathahalli', 'Sarjapur Road', 'Wilson Garden', 'Shanti Nagar',
       'Koramangala 5th Block', 'Koramangala 8th Block', 'Richmond Road',
       'Koramangala 7th Block', 'Jalahalli', 'Koramangala 4th Block',
       'Bellandur', 'Whitefield', 'East Bangalore', 'Old Airport Road',
       'Indiranagar', 'Koramangala 1st Block', 'Frazer Town', 'RT Nagar',
       'MG Road', 'Brigade Road', 'Lavelle Road', 'Church Street',
       'Ulsoor', 'Residency Road', 'Shivajinagar', 'Infantry Road',
       'St. Marks Road', 'Cunningham Road', 'Race Course Road',
       'Commercial Street', 'Vasanth Nagar', 'HBR Layout', 'Domlur',
       'Ejipura', 'Jeevan Bhima

In [7]:
#lenght of unique locations
len(df['location'].unique())

93

In [8]:
#create a new dataframe for locations
locations = pd.DataFrame()

#add column called Name to it
locations['Name'] = df['location'].unique()

#print the head
locations.head()

,Name
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


extracting longitudes and latitudes for the locations

In [9]:
#installing geopy. Geopy is a library that makes it easy to locate coordinates of addresses
!pip install geopy

In [10]:
from geopy.geocoders import Nominatim
#Nominatim uses OpenStreetMap data to find locations on earth

In [11]:
geolocator = Nominatim(user_agent='app')
#assign user_agent to app to establish connection to the nominatim

In [12]:
#define new lists as lat and long and then iterate using for-in loop
#geolocator is used to convert the iterable into some geographical format
lat = []
long = []
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat.append(np.nan)
        long.append(np.nan)
    else:
        lat.append(location.latitude)
        long.append(location.longitude)

In [13]:
#add new columns for long and lat and link them to lat,long
locations['lat']=lat
locations['long']=long

#call head on the data
locations.head()

,Name,lat,long
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.387214,76.666963
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318


#### lets save the locations dataframe as a csv file

In [14]:
locations.to_csv(r'C:\Users\chidiebere\Desktop\Geospatial Data Science in Python\Zomato_cordinates.csv',index=False)

In [15]:
#getting the number of restaurants per location
rest_location=df['location'].value_counts().reset_index()
rest_location

,index,location
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144
...,...,...
88,Yelahanka,6
89,West Bangalore,6
90,Jakkur,3
91,Rajarajeshwari Nagar,2


In [16]:
#change the name of the columns
rest_location.columns=['Name','count']
rest_location

,Name,count
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144
...,...,...
88,Yelahanka,6
89,West Bangalore,6
90,Jakkur,3
91,Rajarajeshwari Nagar,2


In [17]:
#merge rest_locations with locations to get the location alond with the coordinates
restaurant_locations=rest_location.merge(locations,on='Name',how='left').dropna()
restaurant_locations

,Name,count,lat,long
0,BTM,5124,45.954851,-112.496595
1,HSR,2523,18.147500,41.538889
2,Koramangala 5th Block,2504,12.934377,77.628415
3,JP Nagar,2235,12.265594,76.646540
4,Whitefield,2144,44.373058,-71.611858
...,...,...,...,...
88,Yelahanka,6,13.100698,77.596345
89,West Bangalore,6,12.984852,77.540063
90,Jakkur,3,13.078474,77.606894
91,Rajarajeshwari Nagar,2,12.927441,77.515522


### Visualisations and analysis

In [18]:
#for making heat maps
!pip install folium

#### HeatMap of Restaurant density

In [21]:
import folium
basemap=generatebasemap()

In [22]:
#defining function to create a basemap so that I can call the function whenever the basemap is needed
def generatebasemap(default_location=[12.97, 77.59], default_zoom_start=12):
    basemap = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return basemap

In [23]:
basemap

In [24]:
#create a heatmap
from folium.plugins import HeatMap

In [27]:
#create a heatmap and add to basemap
HeatMap(restaurant_locations[['lat','long','count']],zoom=20).add_to(basemap)

In [28]:
basemap

### Restaurants are clustered more in central Bnagaluru area which is no surprise as it happens to be the center of India's high-tech industry known for it's parks and night life. These are ideal environments for restaurants to thrive in this age. It is a metropolis with high standards of living, hence a very wealthy population who can afford a estaurant meal.

### Geo Analysis: where are the restaurants located in Bengaluru using Marker Cluster?

In [29]:
from folium.plugins import FastMarkerCluster

In [32]:
# Plugin: FastMarkerCluster
FastMarkerCluster(data=restaurant_locations[['lat','long','count']].values.tolist()).add_to(basemap)

basemap

#### identifying restaurants with high average rating

In [33]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [34]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [35]:
#there are missing values so we drop them
df.dropna(axis = 0, subset = ['rate'], inplace = True)

In [38]:
#eliminate the separator / from the ratings
def split(x):
    return x.split('/')[0]

In [41]:
#split and add to df
df['rating'] = df['rate'].apply(split)
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [42]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 'NEW', '2.9', '3.5',
       '2.6', '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2',
       '2.3', '3.4 ', '-', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ',
       '3.7 ', '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ',
       '3.2 ', '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8',
       '4.6 ', '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ',
       '2.2 ', '2.0 ', '1.8 '], dtype=object)

In [46]:
#wherever we see 'NEW', we replace with 0
df.replace('NEW',0,inplace = True)

In [48]:
#wherever we see hyphen, we replace with 0
df.replace('-',0, inplace = True)

In [49]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 0, '2.9', '3.5', '2.6',
       '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2', '2.3',
       '3.4 ', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ', '3.7 ',
       '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ', '3.2 ',
       '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8', '4.6 ',
       '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ', '2.2 ',
       '2.0 ', '1.8 '], dtype=object)

### average rating for each location

In [51]:
df.groupby('location')['rating'].mean()

DataError: No numeric types to aggregate

In [52]:
df.dtypes

url                            object
address                        object
name                           object
online_order                   object
book_table                     object
rate                           object
votes                           int64
phone                          object
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
rating                         object
dtype: object

In [55]:
#convert the rating column to numeric
df['rating']=pd.to_numeric(df['rating'])

In [60]:
df.groupby('location')['rating'].mean().sort_values(ascending = False)

location
Lavelle Road             4.042886
St. Marks Road           4.017201
Koramangala 3rd Block    3.978756
Sankey Road              3.965385
Church Street            3.963091
                           ...   
Electronic City          3.041909
Bommanahalli             2.926752
Hebbal                   2.880000
North Bangalore          2.385714
West Bangalore           2.020000
Name: rating, Length: 92, dtype: float64

In [65]:
avg_rating = df.groupby('location')['rating'].mean().sort_values(ascending = False).values
avg_rating

array([4.04288577, 4.01720117, 3.97875648, 3.96538462, 3.96309091,
       3.90151197, 3.90105263, 3.85      , 3.84457237, 3.83965517,
       3.81435185, 3.80740741, 3.79642857, 3.74784206, 3.74055024,
       3.7270073 , 3.72222222, 3.70492958, 3.68801262, 3.68783784,
       3.66823708, 3.66246625, 3.65216942, 3.64      , 3.625     ,
       3.61525029, 3.6075    , 3.60645161, 3.59584871, 3.583174  ,
       3.58095238, 3.56487889, 3.54565217, 3.54139845, 3.52914439,
       3.51111111, 3.50267857, 3.49980952, 3.49891697, 3.48695652,
       3.48406955, 3.47894737, 3.4787234 , 3.47818471, 3.47362637,
       3.47355822, 3.45555556, 3.44615385, 3.44551724, 3.4375    ,
       3.42977099, 3.42238193, 3.41978022, 3.41292591, 3.40053227,
       3.4       , 3.38911917, 3.38554779, 3.38417011, 3.3746988 ,
       3.37329193, 3.36292585, 3.36      , 3.33333333, 3.32358974,
       3.3202381 , 3.3202381 , 3.30983302, 3.29892473, 3.29612767,
       3.2940678 , 3.27927928, 3.278125  , 3.27167674, 3.26393

In [68]:
loc_list = df.groupby('location')['rating'].mean().sort_values(ascending = False).index
loc_list

Index(['Lavelle Road', 'St. Marks Road', 'Koramangala 3rd Block',
       'Sankey Road', 'Church Street', 'Koramangala 5th Block',
       'Cunningham Road', 'Rajarajeshwari Nagar', 'Residency Road',
       'Sadashiv Nagar', 'Koramangala 4th Block', 'Langford Town',
       'Infantry Road', 'Koramangala 7th Block', 'MG Road', 'Race Course Road',
       'Kengeri', 'Seshadripuram', 'Richmond Road', 'Hosur Road',
       'Malleshwaram', 'Koramangala 6th Block', 'Indiranagar', 'Yelahanka',
       'Central Bangalore', 'Jayanagar', 'Koramangala 8th Block',
       'Koramangala', 'Brigade Road', 'New BEL Road', 'Vasanth Nagar',
       'Frazer Town', 'Koramangala 2nd Block', 'Ulsoor', 'Kalyan Nagar',
       'Uttarahalli', 'Yeshwantpur', 'Kammanahalli', 'Shivajinagar',
       'Jalahalli', 'HSR', 'Kanakapura Road', 'Sahakara Nagar', 'Basavanagudi',
       'Kaggadasapura', 'Sarjapur Road', 'Mysore Road', 'City Market',
       'Basaveshwara Nagar', 'Magadi Road', 'Jeevan Bhima Nagar',
       'Rajajinag

In [69]:
ratings = pd.DataFrame()

In [71]:
#define new lists as lat and long and then iterate using for-in loop
#geolocator is used to convert the iterable into some geographical format
lat = []
long = []
for loc in loc_list:
    loc = geolocator.geocode(loc)
    if loc is None:
        lat.append(np.nan)
        long.append(np.nan)
    else:
        lat.append(loc.latitude)
        long.append(loc.longitude)

In [72]:
ratings['locations'] = loc_list
ratings['longitude'] = long
ratings['latitude'] = lat
ratings['avg_ratings'] = avg_rating

In [73]:
ratings

,locations,longitude,latitude,avg_ratings
0,Lavelle Road,-76.373824,40.765284,4.042886
1,St. Marks Road,-84.826086,33.119391,4.017201
2,Koramangala 3rd Block,77.626625,12.927187,3.978756
3,Sankey Road,-121.505644,38.780108,3.965385
4,Church Street,-74.006213,40.717134,3.963091
...,...,...,...,...
87,Electronic City,115.214701,-8.636801,3.041909
88,Bommanahalli,77.623904,12.908945,2.926752
89,Hebbal,77.591900,13.038218,2.880000
90,North Bangalore,77.590082,12.976794,2.385714


In [74]:
#check for missing values
ratings.isna().sum()

locations      0
longitude      1
latitude       1
avg_ratings    0
dtype: int64

In [76]:
#drop missing values
ratings.dropna(inplace = True)

#### extract coordinates and avg temp and pass to the heatmap

In [80]:
HeatMap(ratings[['latitude', 'longitude', 'avg_ratings']]).add_to(basemap)

In [81]:
basemap

### The Bangalore metropolis also has the restaurants with the highest ratings compared to the nrighbouring locations around it. The high ratings must be driven by the heavy competition in the area forcing restaurants to step up to attract customers.

### Analysis with respect to zones

In [85]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [87]:
filter = df['cuisines'] == 'North Indian'
df2 = df[filter]
df2.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.8
50,https://www.zomato.com/bangalore/petoo-banasha...,"276, Ground Floor, 100 Feet Outer Ring Road, B...",Petoo,No,No,3.7/5,21,+91 8026893211,Banashankari,Quick Bites,NaN,North Indian,450,"[('Rated 2.0', 'RATED\n This is a neatly made...",[],Delivery,Banashankari,3.7
72,https://www.zomato.com/bangalore/spicy-tandoor...,"Opposite ICICi Bank, Hanuman Nagar, Banashanka...",Spicy Tandoor,No,No,0,0,+91 8050884222,Banashankari,Quick Bites,NaN,North Indian,150,"[('Rated 4.0', 'RATED\n cost for chicken roll...",[],Delivery,Banashankari,0.0
87,https://www.zomato.com/bangalore/krishna-sagar...,"38, 22nd Main, 22nd Cross, Opposite BDA, 2nd S...",Krishna Sagar,No,No,3.5/5,31,+91 8892752997\r\n+91 7204780429,Banashankari,Quick Bites,NaN,North Indian,200,"[('Rated 1.0', 'RATED\n Worst experience with...",[],Delivery,Banashankari,3.5
94,https://www.zomato.com/bangalore/nandhini-delu...,"304, Opposite Apollo Public School, 100 Feet R...",Nandhini Deluxe,No,No,2.6/5,283,080 26890011\r\n080 26890033,Banashankari,Casual Dining,"Biryani, Chicken Guntur, Thali, Buttermilk, Ma...",North Indian,600,"[('Rated 3.0', 'RATED\n Ididnt like much.\n\n...",[],Delivery,Banashankari,2.6


## analysing different categories of restaurant

In [95]:
#this is the check the number of north indian restaurant in each location and storing in a variable
north_indian = df2.groupby('location')['url'].count().reset_index()
north_indian.columns=['Name','count']
north_indian

,Name,count
0,BTM,274
1,Banashankari,35
2,Banaswadi,9
3,Bannerghatta Road,60
4,Basavanagudi,17
...,...,...
59,"Varthur Main Road, Whitefield",3
60,Vasanth Nagar,12
61,Whitefield,148
62,Wilson Garden,37


In [96]:
#merge ithe north_indian df with location df
north_indian = north_indian.merge(locations,on = 'Name', how = 'left').dropna()
north_indian

,Name,count,lat,long
0,BTM,274,45.954851,-112.496595
1,Banashankari,35,15.887678,75.704678
2,Banaswadi,9,13.014162,77.651854
3,Bannerghatta Road,60,12.938943,77.602170
4,Basavanagudi,17,12.941726,77.575502
...,...,...,...,...
59,"Varthur Main Road, Whitefield",3,12.941466,77.747094
60,Vasanth Nagar,12,12.988721,77.585169
61,Whitefield,148,44.373058,-71.611858
62,Wilson Garden,37,12.948934,77.596827


In [97]:
HeatMap(north_indian[['lat','long','count']],zoom=20,radius=15).add_to(basemap)
basemap

### This is a heatmap of all north indian restaurants

#### automate the operation so whatever category we pass, it returns the heatmap of that category

In [98]:
def Heatmap_zone(zone):
    filter = df['cuisines']==zone
    df2 = df[filter]
    df_zone = df2.groupby('location')['url'].count().reset_index()
    df_zone.columns=['Name','count']
    df_zone = df_zone.merge(locations,on = 'Name', how = 'left').dropna()
    HeatMap(df_zone[['lat','long','count']],zoom=20,radius=15).add_to(basemap)
    return basemap

In [99]:
df['cuisines'].unique()

array(['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai',
       'Cafe, Mexican, Italian', ..., 'Tibetan, Nepalese',
       'North Indian, Street Food, Biryani',
       'North Indian, Chinese, Arabian, Momos'], dtype=object)

In [100]:
Heatmap_zone('Street Food')

In [102]:
Heatmap_zone('Italian')

In [103]:
Heatmap_zone('Nepalese')

In [105]:
Heatmap_zone('Thai')

In [106]:
Heatmap_zone('Mexican')

In [107]:
Heatmap_zone('Cafe')